(N. B. Add extension import call and calls to show() and play() as desired once that's working.)

# rhythmmakertools as a first example of top-down score control

In [ ]:
from abjad import *

In [4]:
from presentation import *

For this demo, we need to import two existing modules: `presentation.py` gives us some convenient typesetting overrides -- it hides the barlines and measure numbers -- and more legibible spacing -- it turns on proportional spacing -- for complex rhythms. `demo.py` gives us a preview function that shows us where we're headed at the end of the demo.

In [5]:
from demo import *

Now that we've imported demo.py, we can run a preview function to see where we're headed in this demo: at the end, we'll have generated a complicated polyphonic rhythmic texture using generators that know about some pattern information:

In [ ]:
show_demo()

But how do we get to the above output? Let's start from the beginning.

You can build from notational elements in an aggregative way (working "bottom-up") in Abjad, but you can also create generator classes that will output leaves based on pattern information (a "top-down" way of working). Trevor's rhythmmakertools module is one example of this way of working, and we'll look at it in this demo.

# example 1

First, we need to understand the relationship between the two concepts: divisions and selections. Divisions are arbitrary timespans that our rhythm maker converts into durated leaves ("selections") in various ways. To see this, we'll look at the simplest kind of rhythmmaker, a NoteRhythmMaker, which creates a tiechain of durated leaves that add up to a corresponding division:

In [ ]:
note_rhythm_maker = rhythmmakertools.NoteRhythmMaker()

We configure the NoteRhythmMaker by passing it some divisions (a list of arbitrary durations), and the NoteRhythmMaker returns selections (tied groups of durated leaves) that correspond to each division:

In [ ]:
divisions = [(3, 8), (5, 4), (1, 4), (13, 16)]

In [ ]:
selections = note_rhythm_maker(divisions)

In [ ]:
for s in selections:
    print(s)

Note the relationship between durations and selections here. While some divisions yield single-symbol selections ((3,8) yields a dotted quarter note and (1,4) a quarter note), some divisions yield tiechains of multiple symbols ((5,4) yields a whole note tied to a quarter note). This solves a fundamental problem of common practice notation: we can represent 1-8 of anything with a single symbol, but 5 of anything must be represented by two symbols. The "selection" abstraction allows us to address the symbolic corrollary of a division as single object in the system.

And of course, now that we have our selections as leaves, we can render these via LilyPond by putting them in a Container and showing the container:

In [ ]:
staff = Staff(selections, context_name='RhythmicStaff')

In [ ]:
show(staff)

But this output looks horrible, because our durations don't fit neatly into LilyPond's defaults of mensural notation in common time. In many notation programs, we would be out of luck: we would be forced to think metrically. But in Abjad, we can override LilyPond's defaults and think freely about rhythms, without meter:

In [ ]:
from presentation.py import *

Now we have a helper function, `make_sketch`, that will apply appropriate overrides to a LilyPond file and show the result: 

In [ ]:
sketch = make_sketch(note_rhythm_maker, divisions)

In [ ]:
show(sketch)

Note that we're now viewing unmetered rhythms in proportional notation and can work freely with rhythm, without any requirement to address meter.

Rhythm makers are reconfigurable. Just pass in a different set of divisions for the corresponding selections:

In [ ]:
divisions_b = [(5, 16), (3, 8), (3, 8), (5, 8), (1, 4)]

In [ ]:
sketch = make_sketch(note_rhythm_maker, divisions_b)

In [ ]:
show(sketch)

And of course anything you can do in Python, you can do to the supplied divisions...

In [ ]:
divisions_b *= 20

In [ ]:
sketch = make_sketch(note_rhythm_maker, divisions_b)

In [ ]:
show(sketch)

...including anything that requires code in another module:

In [ ]:
import random

In [ ]:
random_numerators = [random.randrange(1, 16 + 1) for x in range(100)]

In [ ]:
random_divisions = [(x, 32) for x in random_numerators]

In [ ]:
sketch = make_sketch(note_rhythm_maker, random_divisions)

In [ ]:
show(sketch)

# example 2

In this example, we'll model Medieval isorhythmic practice, as can be found in the music of composers like Machaut and Ockeghem. In this technique of composition, a cycle of durations, a Talea, produces a cyclic pattern of rhythmic durations. We'll still pass in a list of divisions, as we did in example 1, but this time they'll control the durations of beamed groups.

In [ ]:
talea = rhythmmakertools.Talea(
    counts=[1, 2, 3],
    denominator=16,
    )

You can see that the Talea creates a cycle of durations:

In [ ]:
for i in range(20):
    print(i, talea[i])

Next, we give this Talea to a special rhythm maker, a TaleaRhythmMaker:

In [ ]:
talea_rhythm_maker = rhythmmakertools.TaleaRhythmMaker(talea=talea)

Remember what our divisions are?:

In [ ]:
divisions

In [ ]:
sketch = make_sketch(talea_rhythm_maker, divisions)

In [ ]:
show(sketch)

Note that the beaming corresponds to our divisions. Note also that when the duration in the Talea's cycle crosses between beamed groups, we tie across beamed groups by default.

We can refine the output with additional specifiers:

A tie specifier allows us to always tie between beamed groups:

In [ ]:
tie_specifier = rhythmmakertools.TieSpecifier(
    tie_across_divisions=True,
    )

extra counts cyclically inflect tempo by adding more equally spaced events to our divisions:

In [ ]:
extra_counts_per_division = [0, 1, 1]

And a burnish specifier cyclically replaces n counts from the left of each division with the corresponding class, i.e., if we have a class list of [Rest] and a count list of [1,0], we would replace the left-most leaf of the first division with a Rest, do nothing to the following division, replace the left-most leaf of the third division, etc.

In [ ]:
burnish_specifier = rhythmmakertools.BurnishSpecifier(
    left_classes=[Rest],
    left_counts=[1, 0],
    )

Let's see all three of these specifiers work together by configuring our rhythm maker with them:

In [ ]:
talea_rhythm_maker = new(
    talea_rhythm_maker,
    burnish_specifier=burnish_specifier,
    extra_counts_per_division=extra_counts_per_division,
    tie_specifier=tie_specifier,
    )

Remember our divisions:

In [ ]:
divisions

And pass in our divisions to return selections:

In [ ]:
sketch = make_sketch(talea_rhythm_maker, divisions)

In [ ]:
show(sketch)

Note the effect of all three specifiers: the output ties across all beamed groups (tie specifier), the second and third divisions have an extra equally spaced sixteenth note in their spans, resulting in new tuplets (extra counts), and every other division has its left-most leaf replaced by a rest (burnish specifier).

# example 3

Finally, we can use the above in a loop to rapidly prototype polyphonic textures.

In [ ]:
score = Score()

In [ ]:
for i in range(12):
    selections = talea_rhythm_maker(divisions, rotation=i)
    voice = Voice(selections)
    staff = Staff([voice], context_name='RhythmicStaff')
    score.append(staff)
    divisions = sequencetools.rotate_sequence(divisions, 1)

In [ ]:
sketch = make_sketch_lilypond_file(score)

In [ ]:
show(sketch)

Note that we vary the texture here through two pattern rotations: first, we rotate the talea by 1 for each staff, cycling [1,2,3...], [3,1,2...], [2,3,1...] per staff; also, we rotate the list of divisions by one each staff, which changes the way that the selections are beamed.

# conclusion

This is just one example of a set of classes that generate rhythms in a top-down way. Because Abjad extends Python, you can use any of the object-oriented programming capabilities of the Python language to create your own similar maker classes, according to your own musical ideas.